난 이거없으면 시체

In [337]:
import pandas as pd
import numpy as np
import openpyxl
from faker import Faker ## 샘플데이터 생성 라이브러리
fake = Faker('ko-KR')

import collections
from collections import OrderedDict

## 회원 데이터 만들기
- 하나카드 회원수 50만명 가정
- 인구 비율 참고
- 신용카드 가입 비율 참고
- 성비 50:50 가정

- 참고자료
1. 202108_202108_연령별인구현황_월간<br>
2. [2015.4] 우리나라의 지급수단 이용현황 및 선택요인 분석

In [338]:
ratio_df = pd.DataFrame({'population' : [3527385, 3484528, 4164458, 4347302, 5775812], 'creditRatio' : [0.67, 0.979, 0.967, 0.953, 0.828]}, index=[20, 30, 40, 50, 60])

ratio_df['populationRatio'] = ratio_df['population']/ratio_df['population'].sum()

ratio_df['creditCardRatio'] = round(5000*ratio_df['populationRatio'])

ratio_df['creditCardRatio_male']=round(ratio_df['creditCardRatio']/2)
ratio_df['creditCardRatio_female']=round(ratio_df['creditCardRatio']/2)
ratio_df

,population,creditRatio,populationRatio,creditCardRatio,creditCardRatio_male,creditCardRatio_female
20,3527385,0.670,0.165609,828.0,414.0,414.0
30,3484528,0.979,0.163597,818.0,409.0,409.0
40,4164458,0.967,0.195519,978.0,489.0,489.0
50,4347302,0.953,0.204104,1021.0,510.0,510.0
60,5775812,0.828,0.271171,1356.0,678.0,678.0


- 샘플 갯수 (3000명 기준)
- 20대 : 각 414명
- 30대 : 각 409명
- 40대 : 각 489명
- 50대 : 각 510명
- 60대 : 각 678명

In [339]:
### 샘플은 서울시 빅데이터 캠퍼스에서 가져옴

In [340]:
consume2018 = pd.read_csv('data/consume.csv', encoding='utf-8')
member = pd.read_csv('data/member.csv', encoding='utf-8')

In [341]:
## 카테고리 세분화 필요 -> 카드데이터 혜택 중심으로


In [342]:
consume2018['CATEGORY']= consume2018['CATEGORY'].\
replace(['편의점잡화','편의점'], '편의점').\
replace(['기타쇼핑','홈/인터넷쇼핑', '패션','아울렛','육아쇼핑','할인점/슈퍼마켓','전자제품','육아용품'], '쇼핑').\
replace(['마트/슈퍼','백화점'],'대형마트').\
replace(['한식','일반대중음식','패스트푸드','일식','양식','기타식비','부페','중식','술/유흥', '치킨','배달/분식','패밀리R','농수산물','배달'],'외식').\
replace(['버스/기차','유료도로','택시','주차/정비'],'대중교통').\
replace(['여가/놀이','테마파크','기타문화','게임/음악','PC게임방','숙박','인테리어'],'문화').\
replace(['여가스포츠','골프','일상스포츠'],'레저').\
replace(['베이커리','카페','제과점','커피전문점'],'카페/베이커리').\
replace(['이동통신'],'통신').\
replace(['드러그스토어','뷰티로드샵','스파/마사지','헤어','헬스','세탁수선','결혼','애완','악세사리','네일'],'뷰티').\
replace(['약국', '한의원','기타미용','치과','장례','요양원','의료기기','기타건강'],'의료').\
replace(['여행/항공','면세점'],'항공마일리지').\
replace(['렌터카','렌탈']).\
replace(['도서','학원','기타교육/육아','독서실','등록금','보육시설'],'교육').\
replace(['기타공과금','회비','가스요금','보험료','전기요금','세금','관리비'],'관리비')

In [343]:
consume2018['STORE']=consume2018['CATEGORY']+"항목"

In [344]:
def makeTime(x):
    x=str(x)
    if(len(x)==1):
        x="0"+x+":00"
    else:
        x=x+":00"
    return x

In [345]:
consume2018["CONSUME_TIME"] = consume2018["CONSUME_TIME"].apply(makeTime)

In [346]:
consume2018

,MEMBER_NO,GENDER,CONSUME_DATE,CONSUME_TIME,CATEGORY,AMOUNT,STORE
0,Z5563241211ST,M,2018-01-01,00:00,문화,10000,문화항목
1,Z5563241211ST,M,2018-01-01,00:00,편의점,10000,편의점항목
2,Z2963259961ST,M,2018-01-01,00:00,외식,130000,외식항목
3,Z2633227421ST,M,2018-01-01,00:00,쇼핑,130000,쇼핑항목
4,Z2633227421ST,M,2018-01-01,00:00,쇼핑,130000,쇼핑항목
...,...,...,...,...,...,...,...
234847,Z4463263081ST,M,2018-12-31,23:00,카페/베이커리,5000,카페/베이커리항목
234848,Z5224277281ST,M,2018-12-31,23:00,카페/베이커리,5000,카페/베이커리항목
234849,Z6924298191ST,M,2018-12-31,23:00,문화,5000,문화항목
234850,Z0424290721ST,M,2018-12-31,23:00,쇼핑,60000,쇼핑항목


In [347]:
member['NAME']=''
member['NAME']=member['NAME'].apply(lambda x:fake.name())

In [348]:
member['ID']=''
member['ID']=member['ID'].apply(lambda x:fake.email()+str(fake.pyint(min_value=0, max_value=999)))

In [349]:
fake.email()

'ihyeonjun@gimyangseong.net'

In [350]:
member['MOBILE_NO']=''
member['MOBILE_NO']=member['MOBILE_NO'].apply(lambda x:fake.phone_number())

In [351]:
member['RRN']=''
member['RRN']=member['RRN'].apply(lambda x:str(fake.pyint(min_value=600000, max_value=991231))+"-"+str(fake.pyint(min_value=1111111, max_value=2399999)))

In [352]:
member['ISNATIVE']=''
member['ISNATIVE']=member['ISNATIVE'].apply(lambda x:str(fake.random_elements(elements=OrderedDict([("N", 0.99), ('F',0.01) ]), length=1)[0]))

In [353]:
member['PW']=''
member['PW']=member['ISNATIVE'].apply(lambda x:fake.pyint(min_value=100000, max_value=999999))

In [354]:
member['USER_TYPE']='U'

In [355]:
member['ADDRESS']=''
member['ADDRESS']=member['ADDRESS'].apply(lambda x:" ".join(fake.address().split(sep=" ")[0:2]))

In [356]:
member['ADDRESS_DETAIL']=''
member['ADDRESS_DETAIL']=member['ADDRESS_DETAIL'].apply(lambda x:" ".join(fake.address().split(sep=" ")[2:]))

In [357]:
def makeBirthDay(x):
    if(x=='20th'):
        return fake.date_of_birth(tzinfo=None, minimum_age=21, maximum_age=30)
    if(x=='30th'):
        return fake.date_of_birth(tzinfo=None, minimum_age=31, maximum_age=40)
    if(x=='40th'):
        return fake.date_of_birth(tzinfo=None, minimum_age=41, maximum_age=50)
    if(x=='50th'):
        return fake.date_of_birth(tzinfo=None, minimum_age=51, maximum_age=60)
    if(x=='60over'):
        return fake.date_of_birth(tzinfo=None, minimum_age=61, maximum_age=90)

In [358]:
member['BIRTHDAY']=member['AGE_GROUP'].apply(lambda x:makeBirthDay(x))

In [359]:
def makeAge(x):
    if(x=='20th'):
        return fake.pyint(min_value=20, max_value=29)
    if(x=='30th'):
        return fake.pyint(min_value=30, max_value=39)
    if(x=='40th'):
        return fake.pyint(min_value=40, max_value=49)
    if(x=='50th'):
        return fake.pyint(min_value=50, max_value=59)
    if(x=='60over'):
        return fake.pyint(min_value=60, max_value=80)

In [360]:
member['AGE']=''
member['AGE']=member['AGE_GROUP'].apply(lambda x:makeAge(x))

In [361]:
member['PW']=member['PW'].astype(int)

In [362]:
member.rename(columns={'SEX_CCD' : 'GENDER'}, inplace=True)

In [363]:
member=member.sample(frac=1)

In [364]:
member.rename(columns={'level_0' : 'MEMBER_NO2'}, inplace=True)

In [365]:
def amountNormalized(x):
    x=round(np.random.normal(x,x/10),-2)
    return x
    
    

In [366]:
consume2018['AMOUNT']=consume2018['AMOUNT'].apply(lambda x:amountNormalized(x))

In [367]:
consume2018

,MEMBER_NO,GENDER,CONSUME_DATE,CONSUME_TIME,CATEGORY,AMOUNT,STORE
0,Z5563241211ST,M,2018-01-01,00:00,문화,11200.0,문화항목
1,Z5563241211ST,M,2018-01-01,00:00,편의점,10500.0,편의점항목
2,Z2963259961ST,M,2018-01-01,00:00,외식,138400.0,외식항목
3,Z2633227421ST,M,2018-01-01,00:00,쇼핑,133400.0,쇼핑항목
4,Z2633227421ST,M,2018-01-01,00:00,쇼핑,124200.0,쇼핑항목
...,...,...,...,...,...,...,...
234847,Z4463263081ST,M,2018-12-31,23:00,카페/베이커리,4800.0,카페/베이커리항목
234848,Z5224277281ST,M,2018-12-31,23:00,카페/베이커리,4500.0,카페/베이커리항목
234849,Z6924298191ST,M,2018-12-31,23:00,문화,4900.0,문화항목
234850,Z0424290721ST,M,2018-12-31,23:00,쇼핑,65500.0,쇼핑항목


In [368]:
member

,MEMBER_NO,AGE_GROUP,GENDER,HM_ZPN,NAME,ID,MOBILE_NO,RRN,ISNATIVE,PW,USER_TYPE,ADDRESS,ADDRESS_DETAIL,BIRTHDAY,AGE
4715,Z9614216021ST,60over,F,50998,장성현,jinu50@yuhanhoesa.com632,070-8654-5280,760389-2369131,N,361101,U,경기도 부여군,반포대가 (채원김동),1960-06-26,66
4333,Z8753275351ST,60over,F,28456,강순자,gangseonghun@hotmail.com177,062-565-2938,730491-1632186,N,707180,U,부산광역시 강북구,오금336로 (민재김손마을),1931-04-20,80
703,Z1553242741ST,20th,M,4426,이준서,gimsanghun@yu.com401,052-255-4629,620180-1654518,N,111523,U,광주광역시 노원구,양재천15로,1996-10-23,29
475,Z0914229761ST,20th,F,1165,박미숙,kmun@live.com43,032-650-7767,674116-2197908,N,471767,U,광주광역시 서구,서초중앙거리,1999-06-09,21
17,Z0034289451ST,20th,F,38067,김은주,hyeonjuryu@seogim.kr0,061-652-3398,661736-1982308,N,806074,U,세종특별자치시 중구,석촌호수길,1993-11-05,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,Z9534217061ST,60over,F,28674,김현숙,icoe@live.com333,043-735-5970,875906-1753247,N,965343,U,경상남도 하남시,서초중앙가,1935-05-19,60
1400,Z2824289121ST,30th,M,16547,박영숙,yeongsugim@gimjeon.kr221,051-406-2650,737406-2031086,N,869453,U,경상북도 홍천군,학동804가 (진우나김읍),1990-08-20,39
4044,Z8353299921ST,60over,M,22547,이서영,gimyunseo@gmail.com107,061-010-4718,845569-1361926,N,770139,U,서울특별시 성동구,석촌호수길 (현숙허김동),1939-03-23,75
432,Z0824245561ST,20th,F,24326,양준서,kgim@gimi.org74,053-153-5431,730237-1957206,N,176873,U,충청북도 용인시,학동길,1993-06-08,28


In [369]:
member = member[['MEMBER_NO', 'ID', 'PW', 'NAME', 'MOBILE_NO','RRN','BIRTHDAY','AGE','GENDER','ISNATIVE','ADDRESS','ADDRESS_DETAIL']]

In [370]:
member.reset_index(inplace=True)

In [371]:
del member['index']

In [372]:
member.reset_index(inplace=True)

In [373]:
member['index']=member['index']+1

<ipython-input-373-b85ee41e3eda>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member['index']=member['index']+1


In [374]:
card = pd.read_csv('data/card.csv')

In [375]:
card

,CARD_NO,MEMBER_NO,CARD_CODE,CARD_PASSWORD,VALID_DATE,CVC_NO,Unnamed: 6,Unnamed: 7
0,1111-1111-1111-1111,Z0013222301ST,NaN,NaN,NaN,NaN,NaN,NaN
1,1111-1111-1111-1112,Z0013257851ST,NaN,NaN,NaN,NaN,NaN,NaN
2,1111-1111-1111-1113,Z0014223561ST,NaN,NaN,NaN,NaN,NaN,NaN
3,1111-1111-1111-1114,Z0014237071ST,NaN,NaN,NaN,NaN,NaN,NaN
4,1111-1111-1111-1115,Z0014242601ST,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4957,1111-1111-1111-6068,Z9943295311ST,NaN,NaN,NaN,NaN,NaN,NaN
4958,1111-1111-1111-6069,Z9944254381ST,NaN,NaN,NaN,NaN,NaN,NaN
4959,1111-1111-1111-6070,Z9952245691ST,NaN,NaN,NaN,NaN,NaN,NaN
4960,1111-1111-1111-6071,Z9953260751ST,NaN,NaN,NaN,NaN,NaN,NaN


In [376]:
def makeCardCode(x):
    x= str(fake.random_elements(elements=OrderedDict(
        [(1001, 0.05), (1002,0.05), (1003,0.05), (1004,0.05), (1005,0.05),
        (1006, 0.4), (1007, 0.4), (1008, 0.4), (1009, 0.4), (1010, 0.4),
        (1011, 0.3), (1012, 0.3), (1013, 0.3), (1014, 0.3), (1015, 0.3), (1016, 0.3), (1017, 0.3), (1018, 0.3), (1019, 0.3), (1020, 0.3),
         (1021, 0.2), (1022, 0.2), (1023, 0.2), (1024, 0.2), (1025, 0.2), (1026, 0.2), (1027, 0.2), (1028, 0.2), (1029, 0.2), (1030, 0.2),
         (1031, 0.1), (1032, 0.1), (1033, 0.1), (1034, 0.1), (1035, 0.1)
        ]), length=1)[0])
    return x

In [377]:
card['CARD_CODE']=card['CARD_CODE'].apply(makeCardCode)

In [378]:
card['CARD_PASSWORD']=card['CARD_PASSWORD'].apply(lambda x:fake.pyint(min_value=1111, max_value=9999))

In [379]:
card['VALID_DATE']='2025-12-31'

In [380]:
card['CVC_NO']=card['CVC_NO'].apply(lambda x:fake.pyint(min_value=111, max_value=999))

In [381]:
card

,CARD_NO,MEMBER_NO,CARD_CODE,CARD_PASSWORD,VALID_DATE,CVC_NO,Unnamed: 6,Unnamed: 7
0,1111-1111-1111-1111,Z0013222301ST,1010,9484,2025-12-31,310,NaN,NaN
1,1111-1111-1111-1112,Z0013257851ST,1003,4984,2025-12-31,829,NaN,NaN
2,1111-1111-1111-1113,Z0014223561ST,1010,4241,2025-12-31,571,NaN,NaN
3,1111-1111-1111-1114,Z0014237071ST,1014,7437,2025-12-31,425,NaN,NaN
4,1111-1111-1111-1115,Z0014242601ST,1016,7623,2025-12-31,725,NaN,NaN
...,...,...,...,...,...,...,...,...
4957,1111-1111-1111-6068,Z9943295311ST,1008,3355,2025-12-31,212,NaN,NaN
4958,1111-1111-1111-6069,Z9944254381ST,1021,9832,2025-12-31,288,NaN,NaN
4959,1111-1111-1111-6070,Z9952245691ST,1016,3120,2025-12-31,183,NaN,NaN
4960,1111-1111-1111-6071,Z9953260751ST,1010,8379,2025-12-31,410,NaN,NaN


In [382]:
member

,index,MEMBER_NO,ID,PW,NAME,MOBILE_NO,RRN,BIRTHDAY,AGE,GENDER,ISNATIVE,ADDRESS,ADDRESS_DETAIL
0,1,Z9614216021ST,jinu50@yuhanhoesa.com632,361101,장성현,070-8654-5280,760389-2369131,1960-06-26,66,F,N,경기도 부여군,반포대가 (채원김동)
1,2,Z8753275351ST,gangseonghun@hotmail.com177,707180,강순자,062-565-2938,730491-1632186,1931-04-20,80,F,N,부산광역시 강북구,오금336로 (민재김손마을)
2,3,Z1553242741ST,gimsanghun@yu.com401,111523,이준서,052-255-4629,620180-1654518,1996-10-23,29,M,N,광주광역시 노원구,양재천15로
3,4,Z0914229761ST,kmun@live.com43,471767,박미숙,032-650-7767,674116-2197908,1999-06-09,21,F,N,광주광역시 서구,서초중앙거리
4,5,Z0034289451ST,hyeonjuryu@seogim.kr0,806074,김은주,061-652-3398,661736-1982308,1993-11-05,28,F,N,세종특별자치시 중구,석촌호수길
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4957,4958,Z9534217061ST,icoe@live.com333,965343,김현숙,043-735-5970,875906-1753247,1935-05-19,60,F,N,경상남도 하남시,서초중앙가
4958,4959,Z2824289121ST,yeongsugim@gimjeon.kr221,869453,박영숙,051-406-2650,737406-2031086,1990-08-20,39,M,N,경상북도 홍천군,학동804가 (진우나김읍)
4959,4960,Z8353299921ST,gimyunseo@gmail.com107,770139,이서영,061-010-4718,845569-1361926,1939-03-23,75,M,N,서울특별시 성동구,석촌호수길 (현숙허김동)
4960,4961,Z0824245561ST,kgim@gimi.org74,176873,양준서,053-153-5431,730237-1957206,1993-06-08,28,F,N,충청북도 용인시,학동길


In [383]:
card = pd.merge(member, card, on='MEMBER_NO')[['index','CARD_NO','CARD_CODE','MEMBER_NO','CARD_CODE','CARD_PASSWORD','VALID_DATE','CVC_NO']]

In [384]:
card

,index,CARD_NO,CARD_CODE,MEMBER_NO,CARD_CODE,CARD_PASSWORD,VALID_DATE,CVC_NO
0,1,1111-1111-1111-5826,1023,Z9614216021ST,1023,2306,2025-12-31,477
1,2,1111-1111-1111-5444,1025,Z8753275351ST,1025,6159,2025-12-31,723
2,3,1111-1111-1111-1814,1034,Z1553242741ST,1034,9888,2025-12-31,614
3,4,1111-1111-1111-1586,1020,Z0914229761ST,1020,5652,2025-12-31,640
4,5,1111-1111-1111-1128,1020,Z0034289451ST,1020,9134,2025-12-31,630
...,...,...,...,...,...,...,...,...
4957,4958,1111-1111-1111-5733,1005,Z9534217061ST,1005,1584,2025-12-31,460
4958,4959,1111-1111-1111-2511,1011,Z2824289121ST,1011,4993,2025-12-31,176
4959,4960,1111-1111-1111-5155,1010,Z8353299921ST,1010,1303,2025-12-31,399
4960,4961,1111-1111-1111-1543,1008,Z0824245561ST,1008,6843,2025-12-31,804


In [385]:
consume = pd.merge(card, consume2018, on='MEMBER_NO')[['CARD_NO','CONSUME_DATE','CONSUME_TIME','AMOUNT','CATEGORY','STORE']].sort_values('CONSUME_DATE')

In [386]:
consume

,CARD_NO,CONSUME_DATE,CONSUME_TIME,AMOUNT,CATEGORY,STORE
0,1111-1111-1111-1814,2018-01-01,14:00,5200.0,편의점,편의점항목
50362,1111-1111-1111-3647,2018-01-01,09:00,11000.0,의료,의료항목
50363,1111-1111-1111-3647,2018-01-01,09:00,5200.0,대형마트,대형마트항목
50364,1111-1111-1111-3647,2018-01-01,11:00,8400.0,편의점,편의점항목
50365,1111-1111-1111-3647,2018-01-01,20:00,5400.0,카페/베이커리,카페/베이커리항목
...,...,...,...,...,...,...
53934,1111-1111-1111-1270,2018-12-31,13:00,5900.0,쇼핑,쇼핑항목
53933,1111-1111-1111-1270,2018-12-31,08:00,4700.0,편의점,편의점항목
167184,1111-1111-1111-5639,2018-12-31,15:00,10400.0,카페/베이커리,카페/베이커리항목
25713,1111-1111-1111-1973,2018-12-31,19:00,19600.0,카페/베이커리,카페/베이커리항목


In [387]:
member.to_csv('data/result/member.csv', encoding='cp949')
card.to_csv('data/result/card.csv')
consume.to_csv('data/result/consume.csv', encoding='cp949')

In [ ]:
consume.reset_index(inplace=True)
consume.reset_index(inplace=True)

In [ ]:
del consume['index']

In [ ]:
consume['HISTORY_NO'] = consume['level_0']+1

In [ ]:
del consume['level_0']

In [ ]:
del member['MEMBER_NO']

In [257]:
consume

,CARD_NO,CONSUME_DATE,CONSUME_TIME,AMOUNT,CATEGORY,STORE
151038,1111-1111-1111-4229,2018-01-01,10:00,5100.0,쇼핑,쇼핑항목
42700,1111-1111-1111-1943,2018-01-01,05:00,9800.0,편의점,편의점항목
109826,1111-1111-1111-3405,2018-01-01,14:00,43500.0,대형마트,대형마트항목
43403,1111-1111-1111-1956,2018-01-01,00:00,10500.0,문화,문화항목
163459,1111-1111-1111-4496,2018-01-01,03:00,10600.0,편의점,편의점항목
...,...,...,...,...,...,...
180115,1111-1111-1111-4885,2018-12-31,22:00,13200.0,편의점,편의점항목
179940,1111-1111-1111-4878,2018-12-31,18:00,3900.0,카페/베이커리,카페/베이커리항목
179939,1111-1111-1111-4878,2018-12-31,14:00,5400.0,카페/베이커리,카페/베이커리항목
179938,1111-1111-1111-4878,2018-12-31,13:00,11100.0,외식,외식항목


In [256]:
card

,CARD_ENROLL_NO,index,CARD_NO,CARD_CODE,CARD_PASSWORD,VALID_DATE,CVC_NO,MEMBER_NO
0,1,395,1111-1111-1111-1111,1015,9455,2025-12-31,527,Z0814234351ST
1,2,3985,1111-1111-1111-1112,1005,2910,2025-12-31,907,Z8134216611ST
2,3,3861,1111-1111-1111-1113,1035,6244,2025-12-31,608,Z7814239721ST
3,4,4851,1111-1111-1111-1114,1032,1962,2025-12-31,273,Z9724292981ST
4,5,4401,1111-1111-1111-1115,1004,1666,2025-12-31,506,Z8914271481ST
...,...,...,...,...,...,...,...,...
4957,4958,3504,1111-1111-1111-6068,1018,7768,2025-12-31,323,Z7234254701ST
4958,4959,902,1111-1111-1111-6069,1026,8631,2025-12-31,533,Z1814285151ST
4959,4960,1224,1111-1111-1111-6070,1031,9079,2025-12-31,402,Z2634248411ST
4960,4961,3056,1111-1111-1111-6071,1017,7038,2025-12-31,390,Z6353232541ST


fake.address()

fake.name()

fake.pyint(min_value=1111111, max_value=9999999)

fake.date_of_birth(tzinfo=None, minimum_age=21, maximum_age=80)

fake.phone_number()
c
fake.email()

str(fake.pyint(min_value=600000, max_value=991231))+"-"+str(fake.pyint(min_value=1111111, max_value=2399999))

ID, PW, NAME, MOBILE_NO, RRN, BIRTHDAY, AGE, GENDER, ISNATIVE, MANAGER_NO, USER_TYPE, ADDRESS, ADDRESS_DETAIL

fake.pyint(min_value=0, max_value=100)

fake.random_element(elements=('M','F'))

for i in range(5):
    print(fake.random_elements(elements=OrderedDict([("a", 0.45), ("b", 0.35), ("c", 0.15), ("d", 0.05), ]), length=20, unique=False))